# CNN Tutorial with PyTorch on the MNIST Dataset

In this tutorial, we will build a Convolutional Neural Network (CNN) to classify handwritten digits from the MNIST dataset. We'll go through the following steps:

1. **Data Loading, Processing, and Augmentation**
2. **Data Exploration**
3. **Model Building**
4. **Model Training**
5. **Model Evaluation**



## 1. Data Loading, Processing, and Augmentation

The MNIST dataset consists of 28x28 pixel grayscale images of handwritten digits (0-9). We'll use PyTorch's `torchvision` to load and preprocess the data. We will also apply some augmentations to improve our model's robustness.

### Exercise

1. Import the necessary libraries.
2. Load the MNIST dataset using `torchvision.datasets`.
3. Apply normalization to the dataset.
4. Create data loaders for training and testing with appropriate batch sizes.
5. Experiment with different data augmentations like rotation, translation, and flipping.

In [9]:
# ..

In [10]:
# ..

In [16]:
# plot image
import matplotlib_pyplot as plt
from scipy._lib.cobyqa.subsolvers import optim

plt.inshow(train_dataset[4][0].numpy().squeeze(), )

ModuleNotFoundError: No module named 'matplotlib_pyplot'

## 2. Data Exploration

Before training our model, it's essential to explore the data. This helps us understand its distribution and visualize some examples.

### Exercise
1. Visualize a few images from the training dataset along with their labels.
2. Print the number of samples in the training and test datasets.

In [ ]:
# ..

## 3. Model Building

Now, we'll define our CNN architecture. A typical CNN consists of convolutional layers, activation functions, pooling layers, and a fully connected output layer.

### Exercise
1. Define a CNN class inheriting from torch.nn.Module.
2. Include two convolutional layers, ReLU activations, max pooling, and a fully connected layer.

In [18]:
from torch import nn

class CNNModel(nn.Module):
    
    def __init__(self, kernel_size, stride, padding):
        super(CNNModel, self), __init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=kernel_size, stride=stride, padding)
        self.conv2 = nn.Conv2d(10, 20, kernel_size, stride=stride, padding=padding)
        self.relu = nn.ReLU()
        self.avgpool = nn.AvgPool2d(kernel_size=kernel_size, stride=stride)
        self.fc = nn.Linear(kernel_size * kernel_size * 20, 110)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(110, 10)
        self.softmax = nn.Softmax(dim=1)
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return self.softmax(x)
    
    model = CNNModel(kernel_size=3, stride=1, padding=0)
    print(model)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 30)

## 4. Model Training

We'll now define the training loop to optimize our model using the cross-entropy loss function and the Adam optimizer.

### Exercise
1. Define a function to train the model for a specified number of epochs.
2. Print the training loss after each epoch.

In [20]:
import torch 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)
criteron = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

n_epochs = 10

for epoch in range(n_epochs):
    model.train()
    current_loss = 0
    current_accuracy = 0
    for images, label in train_loader:
        image = image.to(device)
        label = label.to(device)
        
        outputs = model(image)
        loss = criteron(outputs, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        current_loss += loss.item()
        
        preds = outputs.data
        _, preds = torch.max(outputs.data, 1)
        
        total += label.size(0)
        correct += (preds == label).sum().items()
        
        
    print(f"Loss: {current_loss / len(train_loader)}")
    print(f"Accuracy: {correct / len(total) *100:.2f")
    
    
        
        

ModuleNotFoundError: No module named 'torch'

## 5. Model Evaluation

After training, we need to evaluate our model on the test dataset to understand its performance.

### Exercise
1. Define a function to evaluate the model's accuracy on the test set.
2. Print the accuracy.

In [13]:
model.eval()

with torch.no_grand():
    for image,label in test_loader:
        image = image.to(device)
        label = label.to(device)
        outputs = model(image)
        _, preds = torch.max(outputs.data, 1)
        
        total += label.size(0)
        correct += (preds == label).sum()
        
    print(f"Accuracy: {(correct / total) * 100:.2f}%")
        
        